In [701]:
import yaml
import os
import sys
import csv
import pystache
import pickle
import re
from pyeda.inter import *
import random
import shutil

In [702]:
#!pip install yaml
#!pip install pystache

In [703]:
# pandoc assignment.md -o assignment.html --mathml -s --metadata title=" "
# pandoc assignment.md -o assignment.pdf -V geometry:margin=1in 

In [704]:
assignment = "/Users/mbs/Google Drive/Courses/ECEM16/2020 Spring/psets/pset1/main.yaml"
basename = os.path.basename(assignment)
dirname_ass = os.path.dirname(assignment)
dirname_dst = "/Users/mbs/Courses/ECEM16/Spring 2020/psets/pset1"
if not os.path.exists(dirname_dst):
    os.makedirs(dirname_dst)
os.chdir(dirname_dst)
output_dirname = os.path.join(dirname_dst,"outputs")
metadata_dirname = os.path.join(dirname_dst,"metadata")

In [705]:
try:
    with open(assignment) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
except IOError as e:
    print(e)
except yaml.scanner.ScannerError as e:
    print("Input file {0} has error: {1}".format(basename,e))
except:
    print("Input file {0} has error: unknown".format(basename))

In [706]:
try:
    problems = data['problems']
except:
    print("Error: Empty assignment as no problems are specified in {0}".format(basename))

if not 'problem_bank' in data:
    problem_bank = os.path.abspath(os.path.join(dirname_dst,"problem_bank"))
    print("Warning: No problem_bank specified in {0}. Using {1}".format(basename,problem_bank))
else:
    problem_bank_dirname = os.path.abspath(os.path.join(dirname_ass,data['problem_bank']))

if not os.path.exists(problem_bank_dirname):
    print("Error: problem bank directory {0} does not exist.".format(problem_bank_dirname))

if not 'roster' in data:
    data['roster'] = "./roster.csv"
    
roster_filename = os.path.abspath(os.path.join(dirname_ass,data['roster']))

if not os.path.exists(roster_filename):
    print("Roster file {0} does not exist.".format(roster_filename))
    roster_present = False
else:
    roster_present = True

In [707]:
probnum = 0
for p in problems:
    probnum += 1
    if not p:
        print("Problem # {0} is not properly specified.".format(probnum))
        print(p)
        assert False, "Halting"
    if not ('file' in p or 'subproblems' in p or 'randompicks' in p):
        print("Problem # {0} has no file or subproblems or randompicks. Skipping.".format(probnum))
        assert False, "Halting"
    if 'file' in p:
        try:
            problem_filename = os.path.join(problem_bank_dirname,p['file'])
            with open(problem_filename) as f:
                p['template'] = (f.read()).strip().rstrip()
        except IOError as e:
            print(e)
        except:
            print("Error: unable to read {0}".format(problem_filename))
        p['directory'] = os.path.dirname(problem_filename)
    elif 'randompicks' in p:
        randompicknum = 0
        for sp in p['randompicks']:
            randompicknum += 1
            if not sp:
                print("Problem # {0} [option {1}] is not properly specified.".format(probnum,randompicknum))
                print(sp)
                assert False, "Halting"
            if not 'file' in sp:
                print("Error: Problem # {0} [option {1}] has no file specified.".format(probnum,randompicknum))
                assert False, "Halting"
            try:
                randompick_filename = os.path.join(problem_bank_dirname,sp['file'])
                with open(randompick_filename) as f:
                    sp['template'] = (f.read()).strip().rstrip()
            except IOError as e:
                print(e)
                assert False, "Halting"
            except:
                print("Error: unable to read {0}".format(randompick_filename))
                assert False, "Halting"
            sp['directory'] = os.path.dirname(randompick_filename)
    else:
        subprobnum = 0
        for sp in p['subproblems']:
            if not sp:
                print("Problem # {0} [option {1}] is not properly specified.".format(probnum,randompicknum))
                print(sp)
                assert False, "Halting"
            if not 'file' in sp:
                print("Error: Problem # {0}_{1} has no file specified.".format(probnum,subprobnum))
                assert False, "Halting"
            subprobnum += 1
            try:
                subproblem_filename = os.path.join(problem_bank_dirname,sp['file'])
                with open(subproblem_filename) as f:
                    sp['template'] = (f.read()).strip().rstrip()
            except IOError as e:
                print(e)
            except:
                print("Error: unable to read {0}".format(subproblem_filename))
            sp['directory'] = os.path.dirname(subproblem_filename)

In [708]:
roster = []
if roster_present:
    with open(roster_filename, newline='') as csvfile:
        roster_reader = csv.reader(csvfile, delimiter=',', 
                                   quotechar='|', skipinitialspace=True)
        for row in roster_reader:
            roster.append([row[0].strip(), row[1].strip().split()[0],row[2].strip().split()[0]])
else:
    roster.append(["", "", ""])

In [709]:
instructions = None

if not 'instructions' in data:
    data['instructions'] = "./instructions.md"
instruction_filename = os.path.abspath(os.path.join(dirname_ass,data['instructions']))
try: 
    with open(instruction_filename) as f:
        instructions = f.read()
except:
    print("Error: unable to read instruction file {0}".format(instruction_filename))

In [710]:
def replexpr(match):
    return str(eval(match.group(1)))
    
def render(source_content, args):
    source_content = str(source_content)
    try:
        content_1 = pystache.render(source_content, args)
    except:
        print("Error in Mustache template:\n{0}\n with parameters\n{1}\n".format(source_content,args))
        import pdb; pdb.set_trace()
        raise SystemExit("Stopping!")
    p = re.compile(r'{%(.*?)%}')
    return p.sub(replexpr, content_1)

In [711]:
def process_problem(problem,args,dirname,dirname_metadata):
    all_content = problem['template'].strip().rstrip()
    yaml_header = {}
    if all_content[:3]=="---":
        i = all_content[4:].find("---")
        if i!=-1:
            try:
                yaml_header = yaml.full_load(all_content[0:i+4])
                all_content = all_content[i+8:].strip()
            except yaml.scanner.ScannerError as e:
                print("Header of problem {0} has error: {1}".format(problem['file'],e))
                #import pdb; pdb.set_trace()
                raise SystemExit("Stopping!")
            except:
                print("Header of problem {0} has error".format(problem['file']))
                #import pdb; pdb.set_trace()
                raise SystemExit("Stopping!")
    #import pdb; pdb.set_trace()
    #assert False
    args_dirname = os.path.join(dirname_metadata,"args")
    if not os.path.exists(args_dirname):
        os.makedirs(args_dirname)
    answerkeys_dirname = os.path.join(dirname_metadata,"answerkeys")
    if not os.path.exists(answerkeys_dirname):
        os.makedirs(answerkeys_dirname)
    args.update(yaml_header.get('defaults',{}))
    args.update(problem.get('params',{}))
    all_content = render(all_content, args)
    answer_key = yaml_header.get('answer_key',[])
    assets_required = yaml_header.get('assets',[])
    for ak in answer_key:
        ak['answer'] = render(ak.get('answer',""), args)
    with open(os.path.join(args_dirname,"args_{0}.pkl".format(args['probnum'])),"wb") as f:
        pickle.dump(args,f)
    if len(answer_key)!=0:
        with open(os.path.join(answerkeys_dirname,"answerkey_{0}.pkl".format(args['probnum'])),"wb") as f:
            pickle.dump(answer_key,f)
    if all_content[-3:]=="```":
        i = all_content[:-3].rfind("```")
        if i!=-1:
            problem_content = all_content[:i]
            answer_sheet = all_content[i:-3]
            i =  answer_sheet.find("#")
            if i!=-1:
                answer_sheet = answer_sheet[i:]
            else:
                answer_sheet = ""
        else:
            problem_content = all_content
            answer_sheet = ""
    else:
        problem_content = all_content
        answer_sheet = "\n# Answer for Problem {0}\n#\n".format(args['probnum'])
    # copy assets if any needed
    if len(assets_required)>0:
        toDirectory = os.path.join(dirname,"assets_{}".format(args['probnum']))
        os.makedirs(toDirectory)
        for asset in assets_required:
            shutil.copy(os.path.join(problem['directory'],"assets/"+asset),toDirectory)
    return problem_content, answer_sheet

In [712]:
if os.path.exists(output_dirname):
    shutil.rmtree(output_dirname)

os.makedirs(output_dirname)

if os.path.exists(metadata_dirname):
    shutil.rmtree(metadata_dirname)

os.makedirs(metadata_dirname) 

problem_variants = {}
for student in roster:
    sid, lastname, firstname = student[0], student[1], student[2]
    problem_variants[sid] = {}
    if roster_present:
        seed = random.seed(sid)
        student_subdirname = "{ln}_{fn}_{sid:01d}".format(
            ln=lastname.lower(),fn=firstname.lower(),sid=int(sid)%10)
        work_dirname = os.path.join(output_dirname,student_subdirname)
        sid_trunc = sid
        while os.path.exists(work_dirname):
            sid_trunc = sid_trunc//10
            student_subdirname = student_subdirname + str(int(sid)%10)
        os.makedirs(work_dirname)
        assignmentfile = os.path.join(work_dirname,"assignment.md")  
        answerfile = os.path.join(work_dirname,"answers.txt") 
        work_dirname_metadata = os.path.join(metadata_dirname,student_subdirname)
        if not os.path.exists(work_dirname_metadata):
            os.makedirs(work_dirname_metadata)
    else:
        seed = random.seed(0)
        work_dirname = output_dirname
        assignmentfile = os.path.join(output_dirname,"assignment.md")
        answerfile = os.path.join(output_dirname,"answers.txt")
        work_dirname_metadata = metadata_dirname
    #image_dirname = os.path.join(work_dirname,"images")
    #if not os.path.exists(image_dirname):
    #    os.makedirs(image_dirname)
    with open(assignmentfile,'w') as f1, open(answerfile,'w') as f2:
        if 'title' in data:
            print("---\ntitle: {0}".format(data['title']),file=f1)
            if 'instructor' in data:
                print("author: {0}".format(data['instructor']),file=f1)
            print("header-includes: |",file=f1)
            print(" \\usepackage{fancyhdr}",file=f1)
            print(" \\pagestyle{fancy}",file=f1)
            print(" \\fancyhead[CO,CE]{Do not distribute.}",file=f1)
            if roster_present:
                print(" \\fancyfoot[CO,CE]{{Created for {0} {1}}}".format(firstname, lastname),file=f1)
            print(" \\fancyfoot[LE,RO]{\\thepage}",file=f1)
            print("---",file=f1)
            print("## {0}".format(data['title']), file=f1)
            print("---", file=f2)
            print("title: '{0}'".format(data['title']), file=f2)
        else:
            print("## Assignment", file=f1)
            print("title: Assignment", file=f2)
        if 'course' in data:
            print("### {0}".format(data['course']), file=f1)
            print("course: '{0}'".format(data['course']), file=f2)
        if 'deadline' in data:
            print("### Due on {0}".format(data['deadline']), file=f1)
            print("deadline: '{0}'".format(data['deadline']), file=f2)
        if roster_present:
            print("### Student Name: {0} {1}".format(firstname, lastname), file=f1)
            print("student: {0} {1}".format(firstname, lastname), file=f2)
            #print("### Student Name: {0} {1} (XXXXX{2})".format(firstname, lastname, sid[-4:]), file=f1)
            #print("student: {0} {1} (XXXXX{2})".format(firstname, lastname, sid[-4:]), file=f2)
        if instructions:
            print("### Instructions:", file=f1)
            f1.write(instructions+"\n")
        print("instructions:", file=f2)
        print("  - 'Do not edit lines starting with # or ---.'", file=f2)
        print("  - You may add lines for your answer.", file=f2)
        print("  - 'Do not start with you answer lines with # or ---.'", file=f2)
        print("---\n", file=f2)
        
        probnum = 0
        for p in problems:
            probnum += 1
            if 'subproblems' in p:
                f1.write("### Problem {0}:".format(probnum))
                #f2.write("---\n# Answer {0}\n#\n".format(probnum))
                if 'title' in p:
                    f1.write(" {}".format(p['title']))
                if 'points' in p:
                    f1.write("    [{} points]".format(p['points']))
                f1.write("\n")
                subproblems = p['subproblems']
                subprobnum = 0
                for sp in subproblems:
                    subprobnum += 1
                    f1.write("#### Subproblem {0}.{1}:".format(probnum, subprobnum))
                    if 'extra_credit' in sp and int(sp['extra_credit'])!=0:
                        f1.write(" Extra Credit:")
                    if 'title' in sp:
                        f1.write(" {}".format(sp['title']))
                    if 'points' in sp:
                        f1.write("    [{} points]".format(sp['points']))
                    f1.write("\n")
                    print("---\n# Answer {0}.{1}\n#".format(probnum,subprobnum), file=f2)
                    args = {'lastname':lastname, 'firstname':firstname, 'sid':sid}
                    args['probnum'] = "{0}_{1}".format(probnum,subprobnum)
                    args['probpoints'] = str(p.get('points',0))
                    args['probfile'] = p.get('file',"")
                    problem_content, answer_sheet = process_problem(sp,args=args, dirname=work_dirname,
                                                                 dirname_metadata=work_dirname_metadata)
                    f1.write(problem_content+"\n")
                    f2.write(answer_sheet+"\n")
            else:
                if 'randompicks' in p:
                    i = random.randrange(len(p['randompicks']))
                    p = p['randompicks'][i]
                    problem_variants[sid][probnum] = i
                    f1.write("### Problem {0} ~[ver{1}]~:".format(probnum,i+1))
                    f2.write("---\n# Answer {0} [ver{1}]\n#\n".format(probnum,i+1))
                else:
                    f1.write("### Problem {0}:".format(probnum))
                    f2.write("---\n# Answer {0}\n#\n".format(probnum))
                if 'extra_credit' in p and int(p['extra_credit'])!=0:
                    f1.write(" Extra Credit:")
                if 'title' in p:
                    f1.write(" {}".format(p['title']))
                if 'points' in p:
                    f1.write("    [{} points]".format(p['points']))
                f1.write("\n")
                args = {'lastname':lastname, 'firstname':firstname, 'sid':sid}
                args['probnum'] = str(probnum)
                args['probpoints'] = str(p.get('points',0))
                args['probfile'] = p.get('file',"")
                problem_content, answer_sheet = process_problem(p,args=args,dirname=work_dirname,
                                                                 dirname_metadata=work_dirname_metadata)
                f1.write(problem_content+"\n")
                f2.write(answer_sheet+"\n")
with open(os.path.join(metadata_dirname,"problem_variants.pkl"),"wb") as f:
    pickle.dump(problem_variants,f)
print("DONE!")

DONE!


In [700]:
# pandoc assignment.md -o assignment.pdf -V geometry:margin=1in 
# pandoc assignment.md -o assignment.html --mathml -s --metadata title=" "

In [665]:
#problem_variants

In [667]:
#pickle.load(open("/Users/mbs/Courses/ECEM16/Spring 2020/psets/pset1/metadata/905419432/answerkeys/answerkey_2.pkl","rb"))

In [666]:
#pickle.load(open("/Users/mbs/Courses/ECEM16/Spring 2020/psets/pset1/metadata/905419432/args/args_2.pkl","rb"))

In [670]:
pickle.load(open(os.path.join(metadata_dirname,"problem_variants.pkl"),"rb"))

{'605391825': {5: 0},
 '405195887': {5: 1},
 '805167463': {5: 1},
 '004997956': {5: 1},
 '404964553': {5: 1},
 '405109333': {5: 0},
 '004975624': {5: 1},
 '105320897': {5: 0},
 '505105424': {5: 0},
 '505394179': {5: 1},
 '305095789': {5: 1},
 '005399315': {5: 0},
 '805193037': {5: 0},
 '905343772': {5: 0},
 '505311773': {5: 0},
 '805314001': {5: 0},
 '205124669': {5: 1},
 '905116878': {5: 1},
 '705346069': {5: 0},
 '905134660': {5: 1},
 '805305940': {5: 1},
 '705340449': {5: 0},
 '405320075': {5: 0},
 '605146971': {5: 0},
 '005205770': {5: 0},
 '905161154': {5: 1},
 '605394522': {5: 1},
 '605283411': {5: 0},
 '605140573': {5: 0},
 '805342674': {5: 1},
 '105116127': {5: 0},
 '905203286': {5: 0},
 '204645920': {5: 0},
 '905303766': {5: 0},
 '105306580': {5: 1},
 '605354941': {5: 1},
 '205409182': {5: 1},
 '505321159': {5: 0},
 '605415001': {5: 0},
 '805305879': {5: 1},
 '705420506': {5: 0},
 '705310999': {5: 0},
 '205300889': {5: 1},
 '305140447': {5: 1},
 '604892225': {5: 1},
 '20530969